# 使用直接偏好优化（DPO）进行偏好对齐

本教程将会带你使用直接偏好优化（DPO）的方法去微调一个语言模型。这里我们使用的是 SmolLM2-135M-Instruct 模型，它已经经过了 SFT 这一训练步骤，所以可以接着进行 DPO。当然你也可以使用你在[第一章](../../1_instruction_tuning/notebooks/sft_finetuning_example_cn.ipynb)训练好的模型。

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
     <h2 style='margin: 0;color:blue'>练习：用 DPOTrainer 对 SmolLM2 做对齐训练</h2>
     <p>从 Hugging Face hub 上找一个数据集并在上面对齐模型。</p> 
     <p><b>难度等级</b></p>
     <p>🐢 使用 `trl-lib/ultrafeedback_binarized` 数据集</p>
     <p>🐕 使用 `argilla/ultrafeedback-binarized-preferences` 数据集</p>
     <p>🦁 选用你感兴趣的真实世界数据集，或者使用你在<a href="../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb">上一章</a> SFT 训练好的模型</p>
</div>

In [ ]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login

login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

## 准备好需要的 Python 库


In [ ]:
import torch
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import DPOTrainer, DPOConfig


## 配置数据集及其格式

In [ ]:
# Load dataset

# TODO: 🦁🐕 你也可以使用你感兴趣的其他数据集
dataset = load_dataset(path="trl-lib/ultrafeedback_binarized", split="train")

In [ ]:
# TODO: 🐕 如果你的数据集不是对话列表形式，你可以使用 `process_dataset` 函数进行转化

## 选择一个经 SFT 训练过模型

我们这里使用 SmolLM2-135M-Instruct 模型，因为它已经经过 SFT 了，所以可以接着进行 DPO 训练。你也可以使用你[第一章](../../1_instruction_tuning/notebooks/sft_finetuning_example_cn.ipynb) 训练好的模型。


<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; width:80%; color:black'>
     <p>🦁 如果你想使用<a href="../../1_instruction_tuning/notebooks/sft_finetuning_example_cn.ipynb">第一章</a>你自己训练的模型，你需要修改下面代码的 model_name 到指定的路径</p>
</div>


In [ ]:
model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

# Model to fine-tune
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    torch_dtype=torch.float32,
).to(device)
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-DPO"
finetune_tags = ["smol-course", "module_1"]

## 开始 DPO 训练

In [ ]:
# Training arguments
training_args = DPOConfig(
    # 单 GPU 上 batch size 的大小
    per_device_train_batch_size=4,
    # 每经过多少步才进行一次反向传播、参数更新
    # 每个 GPU 实际起作用的 batch size 等于 per_device_train_batch_size 乘 gradient_accumulation_steps
    gradient_accumulation_steps=4,
    # 这个操作在前向传播时不保存中间结果激活值，可以节省内存
    # 在反向传播时，模型重新计算前向过程，获取计算梯度所需要的激活值
    gradient_checkpointing=True,
    # 训练时的基本学习率
    learning_rate=5e-5,
    # 训练过程中学习率变化策略："cosine" 策略根据余弦函数的曲线形状逐步递减学习率
    lr_scheduler_type="cosine",
    # 总的训练部数
    max_steps=200,
    # 在模型训练过程中不去保存 ckeckpoint
    save_strategy="no",
    # 打印 log 的频率
    logging_steps=1,
    # 保存结果的目录
    output_dir="smol_dpo_output",
    # 学习率 warmup 的步数
    warmup_steps=100,
    # 是否使用 bfloat16 精度来加速训练
    bf16=True,
    # 这里我们不使用 wandb 或 tensorboard 
    report_to=None,
    # 这里我们保留数据集里没用到的数据列
    remove_unused_columns=False,
    # 如果你不使用 Mac 电脑，这里就是 False
    use_mps_device=device == "mps",
    # 如果你要将训练好的模型上传到 HuggingFace Hub，这里就是你的模型名字 
    hub_model_id=finetune_name,
)

In [ ]:
trainer = DPOTrainer(
    # The model to be trained
    model=model,
    # Training configuration from above
    args=training_args,
    # Dataset containing preferred/rejected response pairs
    train_dataset=dataset,
    # Tokenizer for processing inputs
    processing_class=tokenizer,
    # DPO 特有的 temperature 参数，控制参考模型的强度
    # 较小的值可以让模型更保守、更缓慢地学习偏好
    beta=0.1,
    # 提示语的最大长度
    max_prompt_length=1024,
    # 提示语 + 回答 的序列最大长度
    max_length=1536,
)

In [ ]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

# Save to the huggingface hub if login (HF_TOKEN is set)
if os.getenv("HF_TOKEN"):
    trainer.push_to_hub(tags=finetune_tags)

## 💐 完成

本教程带你一步步地学习了用 `DPOTrainer` 训练 `HuggingFaceTB/SmolLM2-135M`的过程。利用这些步骤，你可以高效快速地进行特定任务领域的偏好优化训练。感兴趣的话，你还可以：

- 尝试本教程难度等级更高的任务
- 在 GitHub 上评审别人的 Pull Request
- 通过 Issue 或 PR 提出课程改进的意见